In [1]:
from gurobipy import *
import math


In [2]:

def distance(a,b):
  dx = a[0] - b[0]
  dy = a[1] - b[1]
  return math.sqrt(dx*dx + dy*dy)

# Problem data
clients = [[0, 1.5],[2.5, 1.2]]
facilities = [[0,0],[0,1],[0,1],
              [1,0],[1,1],[1,2],
              [2,0],[2,1],[2,2]]
charge = [3,2,3,1,3,3,4,3,2]

numFacilities = len(facilities)
numClients = len(clients)

m = Model()

# Add variables
x = {}
y = {}
d = {} # Distance matrix (not a variable)
alpha = 1

for j in range(numFacilities):
  x[j] = m.addVar(vtype=GRB.BINARY, name="x%d" % j)

for i in range(numClients):
  for j in range(numFacilities):
    y[(i,j)] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="t%d,%d" % (i,j))
    d[(i,j)] = distance(clients[i], facilities[j])

m.update()

# Add constraints
for i in range(numClients):
  for j in range(numFacilities):
    m.addConstr(y[(i,j)] <= x[j])

for i in range(numClients):
  m.addConstr(quicksum(y[(i,j)] for j in range(numFacilities)) == 1)

m.setObjective( quicksum(charge[j]*x[j] + quicksum(alpha*d[(i,j)]*y[(i,j)]
                for i in range(numClients)) for j in range(numFacilities)) )

m.optimize()

Academic license - for non-commercial use only
Optimize a model with 20 rows, 27 columns and 54 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 20 rows, 27 columns, 54 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)

Root relaxation: objective 4.723713e+00, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       4.7237129    4.72371  0.00%     -    0s

Explored 0 nodes (15 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 4.72371 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.723712908962e+00, best bound 4.723712908962e+00, gap 0.0000%
